In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_layer = 4

cpu


In [9]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [10]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[57, 68, 67,  5, 73,  1, 73, 61],
        [58, 67,  1, 29, 74, 71, 58, 64],
        [66,  1, 73, 61, 58, 66,  1, 59],
        [67, 58, 78, 62, 67, 60,  1, 73]])
targets:
tensor([[68, 67,  5, 73,  1, 73, 61, 62],
        [67,  1, 29, 74, 71, 58, 64, 54],
        [ 1, 73, 61, 58, 66,  1, 59, 68],
        [58, 78, 62, 67, 60,  1, 73, 61]])


In [12]:

x = train_data[:block_size]
y = train_data[block_size:]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When input is', context,'target is', target)

When input is tensor([80]) target is tensor(44)
When input is tensor([80,  0]) target is tensor(32)
When input is tensor([80,  0,  1]) target is tensor(49)
When input is tensor([80,  0,  1,  1]) target is tensor(1)
When input is tensor([80,  0,  1,  1, 28]) target is tensor(25)
When input is tensor([80,  0,  1,  1, 28, 39]) target is tensor(38)
When input is tensor([80,  0,  1,  1, 28, 39, 42]) target is tensor(28)
When input is tensor([80,  0,  1,  1, 28, 39, 42, 39]) target is tensor(1)


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class GPTLanguagaeModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    
    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguagaeModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype =torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


G-lgt21RTmUSF!FN0x!"D﻿!Ix[eHG_QS8H1Zh JyqNQ*XXuAKFE_(z﻿5'5"(Gy,aifgJPJ*'gG9nmm5z(?we[RQ&H6WT&1
PJP1&_c4(Z9i
KwVGvkBpA1*KXnozt*xQ Y﻿-j:ru-p'b-1y,tyz[EO'vtQ)XNsIFubC]svwKwvu[cnEtJ8HfvKeW&&GbYv(RTvEoqe1v;x4I6?5gEKi)tyRTT&E674QQmt0Vv?FUq1g
ME6OtlgWD;:rQ?&-;ZaCSc﻿ha(FPh]
P5?7P])dx:KQEMZ!"tc9 Yv([eAa3!!OWO?GYz,(7(_WII6RDN0y,fQnJ&
QoviM6Jx?u8He1"?JI4jmh﻿.v(NME'C&_;XNv*WGiuT'"V0MMMSZRfgqql,2PJe7rE4Tn2wF"09nfOu7!JIxUYAHnYePGCko!L.cHNH.0M.v5EfQ"]su-1"0hfN7RQWayPriGPGpAX4W3k&cFZR(﻿5zXjruqKp*?_;jXUQbCTq1DeA


In [15]:
#Creating a PyTorch Optimizer

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters ==0:
        losses = estimate_loss()
        print(f"step:{iter}, train loss: {losses['train'] : .4f}, val loss: {losses['val'] : .4f}")
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

step:0, train loss:  4.9015, val loss:  4.9056
step:250, train loss:  4.8240, val loss:  4.8394
step:500, train loss:  4.7697, val loss:  4.7807
step:750, train loss:  4.7193, val loss:  4.7251
4.381287097930908


In [16]:
context = torch.zeros((1,1), dtype = torch.long,  device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Q[9G9Dy﻿5f::Y9larEB9V7O!Lol2hgMj-fQ5?yjKGsF]tm&1AZ-0E?dz*Pb5qkPK6i&eZe&ubmulWPJO*.v(8qqWr!"fl1g7:v(O:,C7K&Scn1CliDn6'.WX"4cs?z0X(AcU.ygF!k)*-MEKeid_V?K﻿9y_rEH0*eBmQ )x.YiV TR0AZy;0;x﻿.FfRHs"AHGfr3F6w:r;K8]c.cB'!Hl4kldix!
QVgFm8FQWq1H_r-eetYjTvxQ4y ,i,mwAdneWE'E'fg3MOWo &E;sSFp]lnO)8aoc"8'(eA]8f9TjUP7z(byQzRgO*XzVLT[;6RTD(N753S
Gl6z.tf6.YMQD﻿
*ngq2nVfgiEMfv5'bY["WqxPSWij'4csXPh&e N﻿3b"4;zQDG2B9PJ a*h9?5H0QCR6nW"?bY﻿V,y:hgGrMu9&Jl,gAl1"De
N0rurHffm7j_"vZD﻿RTj4Vbf0!BjC8HrdB.v(PJS]D﻿)",Xi:7gSXz0A
